### 1. Import Data

* Our data was collected from Kaggle.
* Dataset contains information about flight booking options from the website Easemytrip for flight travel between India's top 6 metro cities.
* There are 300261 datapoints and 11 features in the cleaned dataset.

In [ ]:
# Create dataframe using csv data
flights_df = pd.read_csv("/kaggle/input/flight-price-prediction/Clean_Dataset.csv", index_col=0)

# Display first 5 rows.
flights_df.head(5)

In [ ]:
# Collect information about dataframe
flights_df.info()

## 2. Data Preprocessing

#### **2.1 Missing Values**

In [ ]:
# Check dataframe for missing values.
flights_df.isnull().sum()

* We seem to have 0 missing values.

#### **2.2 Detect Outliers**

In [ ]:
# Create boxplots to check for outliers
fig, ax = plt.subplots(1,3, figsize=(14,4))

sns.boxplot(x=flights_df['duration'], ax=ax[0])
ax[0].set_title("Flight Duration Boxplot")
ax[0].set_xlabel("Flight Duration")

sns.boxplot(x=flights_df['days_left'], ax=ax[1])
ax[1].set_title("Days Left Boxplot")
ax[1].set_xlabel("Days Before Flight")

sns.boxplot(x=flights_df['price'], ax=ax[2])
ax[2].set_title("Price Boxplot")
ax[2].set_xlabel("Flight Price")

In [ ]:
def find_outliers(feature_name):
    """
    This function returns the uppper and lower limit of a feature, using IQR
    """
    percent_25 = flights_df[feature_name].quantile(.25)
    percent_75 = flights_df[feature_name].quantile(.75)

    iqr = percent_75 - percent_25

    upper_limit = percent_75 + 1.5 * iqr
    lower_limit = percent_25 - 1.5 * iqr
    
    return lower_limit, upper_limit

In [ ]:
# Investigate Flight Duration outliers.
flights_df['duration'].describe()

In [ ]:
# Take a look at flights with duration time outliers
flights_df[flights_df['duration'] >= find_outliers('duration')[1]]

In [ ]:
# Find percentage of entries that are outliers
dur_outlier = (flights_df[flights_df['duration'] >= find_outliers('duration')[1]].shape[0] / flights_df.shape[0]) * 100
print(f"{round(dur_outlier, 4)}% of entries have an outlier for 'Duration'")

In [ ]:
# Group by number of stops
flights_df.groupby(['stops'])['duration'].max()

* It seems that the outliers in duration are a result of making layover stops between flights.
* We will not remove outliers since overnight layovers and flight cancelations occur naturally.

In [ ]:
# Investigate Price outliers
flights_df['price'].describe()

In [ ]:
# Take a look at flights with price outliers
flights_df[flights_df['price'] >= find_outliers('price')[1]]

In [ ]:
# Find percentage of entries that are outliers
price_outlier = (flights_df[flights_df['price'] >= find_outliers('price')[1]].shape[0] / flights_df.shape[0]) * 100
print(f"{round(price_outlier, 4)}% of entries have an outlier for 'Price'")

In [ ]:
flights_df.groupby(['class'])['price'].max()

* We will keep the outliers for price, because price surges occur naturally.

#### **2.3 Categorical Variables**

In [ ]:
# Get a list of all non numerical features
cols = flights_df.columns
num_cols = flights_df._get_numeric_data().columns
cat_cols = list(set(cols) - set(num_cols))
print(cat_cols)

In [ ]:
# Find all values for destination_city
flights_df['destination_city'].value_counts()

In [ ]:
# Find all values for source_city
flights_df['source_city'].value_counts()

In [ ]:
# Label Encode our Ordianl Variables
flights_df['stops'] = flights_df['stops'].replace({'zero':0, 'one':1, 'two_or_more':2})
flights_df['class'] = flights_df['class'].replace({'Economy':0, 'Business':1})
flights_df.head()

In [ ]:
# Use One-Hot-Encoding for other Categorical Vars.
dummy_vars = ['airline', 'source_city', 'departure_time', 'arrival_time', 'destination_city']
dummies = pd.get_dummies(flights_df[dummy_vars], drop_first=True)
flights_df_2 = pd.concat([flights_df, dummies], axis=1)
flights_df_2.columns

In [ ]:
flights_df_2 = flights_df_2.drop(columns=['airline', 'source_city', 'departure_time', 'arrival_time', 'destination_city'])

## 3. Exploratory Data Analysis

In [ ]:
# Get size and shape of our dataframe
flights_df.info()

* Our dataframe consists of 300153 rows and 11 columns.
* Most of our data is not numerical.

In [ ]:
# Get summary statitstics for our dataframe
flights_df.describe()

* Our duration feature is slightly right skewed distribution.
* Price is highly skewed to the right, so most of the values lie in left tail.
* Standard Deviation for price is high so there is  greater dispersion in data points.

In [ ]:
# Visualize our target variable price.
sns.histplot(flights_df['price'], kde=True, )
plt.show()

* Our target variable has a right skew.
* Most of our price values in a range of 1000-40000.

In [ ]:
sns.heatmap(flights_df.corr(),annot=True)

In [ ]:
# Explore relationship between price and duration
sns.scatterplot(x='duration', y='price', data=flights_df)

#### **QUESTION: Does Price vary with ariline?**

In [ ]:
sns.boxplot(x='airline', y='price', data=flights_df)
plt.title('Price per Airline')

* **Answer: It seems that Vistara and Air India have a higher price range.**

#### **QUESTION: How are ticket prices affected when tickets are bought 1 or 2 days before flight?**

In [ ]:
plt.figure(figsize=(20,6))
g = sns.lineplot(x='days_left', y='price', data=flights_df, marker='o')
g.set_xticks(range(1,51, 1))
plt.show()

* **Answer: On average ticket prices are the highest when bought 2 days before a flight**

#### **QUESTION: Does ticket price change based on departure or arrival time?**

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(20,8))

sns.boxplot(x='arrival_time', y='price', data=flights_df, ax=ax[0,0])
ax[0,0].set_title('Arival Time vs Price')
ax[0,0].set_xlabel('Arrival Time')

sns.barplot(x='arrival_time', y='price', data=flights_df, estimator='median', ax=ax[0,1])
ax[0,1].set_title('Medain Price per Arrival Time')
ax[0,1].set_xlabel('Arrival Time')

sns.boxplot(x='departure_time', y='price', data=flights_df, ax=ax[1,0])
ax[1,0].set_title('Departure Time vs Price')
ax[1,0].set_xlabel('Departure Time')

sns.barplot(x='departure_time', y='price', data=flights_df, estimator='median', ax=ax[1,1])
ax[1,1].set_title('Medain Price per Departure Time')
ax[1,1].set_xlabel('Departure Time')

fig.tight_layout()
plt.show()

**Answers:**
* **Early Morning arrivals seem to have a lower price range and median price overall**
* **Late night arrival have the smallest price range and the lowest median price overall**
* **Late night departures alsos have the smalles price range and smalles median price**

#### **QUESTION: How does ticket price vary between Economy and Business Class ?**

In [ ]:
g = sns.boxplot(x='class', y='price', data=flights_df)
g.set_title('Economy vs Business Class Price Range')
g.set_xticklabels(['Economy', 'Business'])
g.set_xlabel('Flight Class')

* **Answer: It is clear that prices are higher in Business Class**

## 4. Data Split/Selection

In [ ]:
# Drop the features that will not be used
flights_df_2 = flights_df_2.drop(columns=['flight'])

In [ ]:
flights_df_2.columns

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score

X = flights_df_2.drop(['price'], axis=1)
y= flights_df_2['price']

In [ ]:
# Split data into test and train sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

In [ ]:
X_train

## **5. Build Model**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Build the model
lm = LinearRegression()
lm.fit(X_train, y_train)

lm.intercept_
lm.coef_

y_pred = lm.predict(X_test)

In [ ]:
lm.coef_

In [ ]:
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE: {np.sqrt(mean_absolute_error(y_test, y_pred))}")
print(f"R^2: {r2_score(y_test, y_pred)}")

In [ ]:
# Build model using Statsmodel OLS
import statsmodels.api as sm

ols = sm.OLS(y_train, sm.add_constant(X_train)).fit()

ols.summary()